In [1]:
from textblob import TextBlob
import math
from collections import Counter, defaultdict
import tweepy as tw
import json
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [2]:
api_key = "HAcIeikl6eRheQP1oKYGDwPx0"
api_secret = "gEWmqG7QVXzudnviUXDo98L2UKGYg9PNqy1bAR1geClkHjqBGk"
access_token = "1481095074465579008-1pApYlU4HRJWpbAXcbdVEVGzdACuEz"
access_token_secret = "htgD81H4mOxXw5MglAljFRHhcfGAuwsNomXVqvGw0Obwe"
auth = tw.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth,wait_on_rate_limit=True)

In [ ]:
number_of_tweets = 500
tweets = []
user_id =[]
created_at = []
keyword_list = ['News', 'Stock'] # , 'Customers', 'Employees', 'Company', 'Price', 'Shareholders', 'Market', 'Soaring', 'Trends'
company = 'Amazon'
final_kw_list = []

# Crawl data from keywords
for i in range (len(keyword_list)):
    kw = str(company + ' ' + keyword_list[i])
    print(kw)
    final_kw_list.append(kw)
    for i in tw.Cursor(api.search, q = kw , lang ="en", since = "2022-01-14",tweet_mode = "extended").items(number_of_tweets):
        tweets.append(i.full_text)
        user_id.append(i.user.id)
        created_at.append(i.created_at)

Amazon News


In [ ]:
df = pd.DataFrame({"Created_at": created_at,"User_id": user_id,"Tweets":tweets})
df

In [ ]:
df = df[~df.Tweets.str.contains("RT")]
df = df.reset_index(drop = True)

In [ ]:
def clean_up_tweet(text):
    text = re.sub(r'@[A-Za-z0-9_]+','',text) # Remove mentions @
    text = re.sub(r'#', '', text) # Remove hastag symbol
    text = re.sub(r'https?:\/\/[A-Za-z0-9\.\/]+', '', text) # Remove hyper link
    text = re.sub('[^A-Za-z0-9]+', ' ', text) # Remove special characters
    text = re.sub(r'[0-9]+', '', text)
    #Remove stop words
    res = ''
    for word in text.split():
        if word not in stopwords.words('english'):
            res += WordNetLemmatizer().lemmatize(word) + ' '

    return res

In [ ]:
df['Tweets'] = df['Tweets'].apply(clean_up_tweet)
df = df.drop_duplicates(subset='Tweets')
df = df.reset_index(drop=True)
df

In [ ]:
# Create function to get subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

# Create function to get polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

# Function compute negative, positive
df['Subjectivity'] = df['Tweets'].apply(getSubjectivity)
df['Polarity'] = df['Tweets'].apply(getPolarity)

df = df.reset_index(drop=True)
df

In [ ]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
c_vec = CountVectorizer(stop_words=stoplist, ngram_range=(2,3))
# matrix of n_grams
ngrams = c_vec.fit_transform(df['Tweets'])
# count frequency of ngrams
count_values = ngrams.toarray().sum(axis=0)
# list of ngrams
vocab = c_vec.vocabulary_
df_ngram = pd.DataFrame(sorted([(count_values[i],k) for k,i in vocab.items()], reverse=True)
            ).rename(columns={0: 'Frequency', 1:'n-gram'})

df_ngram = df_ngram.drop_duplicates(subset='n-gram')
df_ngram = df_ngram.reset_index(drop=True)
df_ngram['Polarity'] = df_ngram['n-gram'].apply(lambda x: TextBlob(x).polarity)
df_ngram['Subjective'] = df_ngram['n-gram'].apply(lambda x: TextBlob(x).subjectivity)
df_ngram.drop(df_ngram.loc[df_ngram['Polarity'] == 0.0].index, inplace=True)
df_ngram = df_ngram.reset_index(drop=True)
df_ngram

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

y = df['Polarity'].values.astype('float')

X_train, X_test, y_train, y_test = train_test_split(df["Reviews"], y, random_state=42, test_size=0.2)




In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(min_df= 5, ngram_range=(2,3)).fit(X_train)
X_train_vectorized = vect.transform(X_train)

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train_vectorized, y_train)
predictions = model.predict(vect.transform(X_test))
print(predictions)

In [ ]:
# from sklearn.metrics import classification_report
# import numpy as np
# from sklearn import linear_model
# from sklearn import svm
#
# classifiers = [
#     svm.SVR(),
#     linear_model.SGDRegressor(),
#     linear_model.LinearRegression()]
#
# trainingData    = X_train
# trainingScores  = y_train
# predictionData  = X_test
#
# for item in classifiers:
#     print(item)
#     clf = item
#     clf.fit(trainingData, trainingScores)
#     print(y_test)
#     print(clf.predict(predictionData),'\n')